# Sesión 6 Introducción a ETL

+ Extraer
+ Transformar
+ Cargar

## Parte 1 
(clase 5)

In [1]:
import pandas as pd
import numpy as np

In [2]:
def extraer_de_csv(archivo_procesar):
    df = pd.read_csv(archivo_procesar)
    return df

In [3]:
def extraer_de_json(archivo_procesar):
    df = pd.read_json(archivo_procesar, lines = True)
    return df

In [4]:
import xml.etree.ElementTree as et
import glob

In [5]:
def extraer_de_xml(archivo_procesar):
    xtree = et.parse(archivo_procesar)
    xroot = xtree.getroot()
    df = pd.DataFrame(columns = ['nombre','ventas1','ventas'])
    for persona in xroot:
        nombre = persona.find('nombre').text
        ventas1 = float(persona.find('ventas1').text)
        ventas = float(persona.find('ventas').text)
        df = df.append({'nombre':nombre,'ventas1':ventas1,'ventas':ventas}, ignore_index = True)
    return df

In [6]:
def extraer():
    extraer_dato = pd.DataFrame(columns = ['nombre','ventas1','ventas'])
    
    for a_csv in glob.glob('./Archives/*.csv'):
        print(a_csv)
        extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
        
    for a_json in glob.glob('./Archives/*.json'):
        print(a_json)
        extraer_dato = extraer_dato.append(extraer_de_json(a_json), ignore_index = True)
        
    for a_xml in glob.glob('./Archives/*.xml'):
        print(a_xml)
        extraer_dato = extraer_dato.append(extraer_de_xml(a_xml), ignore_index = True)
    return extraer_dato

## Parte 2

## Transformar

In [7]:
def transform(data):
    #Cambiamos de moneda
    data['ventas1'] = round(data.ventas1 * 3.98, 2)
    data['ventas'] = round(data.ventas * 3.98, 2)
    
    #Unimos los mismos nombres
    data1 = data.groupby("nombre").ventas1.sum()
    data2 = data.groupby("nombre").ventas.sum()
    data_new = {"ventas1": data1, "ventas": data2} 
    data_new2 = pd.concat(data_new, axis = 1)
    
    #Sumamos las columnas ventas1 y ventas en una nueva columna
    data_new2['suma'] = data_new2[['ventas1','ventas']].astype(float).sum(axis=1)
    return data_new2

In [8]:
extraer().to_csv('./Data1/data.csv')
d = './Data1/data.csv'
data3 = pd.read_csv(d)

./Archives\source1.csv


C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fut

./Archives\source2.csv
./Archives\source3.csv
./Archives\source1.json
./Archives\source2.json
./Archives\source3.json
./Archives\source1.xml
./Archives\source2.xml
./Archives\source3.xml


C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_xml(a_xml), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1923074529.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'nombre':nombre,'ventas1':ventas1,'ventas':ventas}, ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1923074529.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'nombre':nombre,'ventas1':ventas1,'ventas':ventas}, ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1923074529.py:9: FutureWarning: The frame.append method is deprecated and will be removed 

## Cargar

In [9]:
import sqlalchemy  

In [10]:
def load(df):
    username = 'root'
    hostname = 'localhost'
    database = 'datawarehouse'
    port     = '3306'
    password = '12345'
    con = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(username, password, 
                                                      hostname, database))
    
    df['id'] = 0
    df.to_sql(con=con, name='ventas_final', if_exists='append')

In [11]:
from datetime import datetime

In [12]:
now = datetime.now() 
print('Proceso Comienza',now)
extracted_data = extraer()
transformed_data = transform(data3)
load(transformed_data)
now = datetime.now() 
print('Proceso Finaliza',now)

Proceso Comienza 2022-11-08 19:11:04.080048
./Archives\source1.csv
./Archives\source2.csv
./Archives\source3.csv
./Archives\source1.json
./Archives\source2.json
./Archives\source3.json
./Archives\source1.xml


C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fut

./Archives\source2.xml
./Archives\source3.xml


C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1923074529.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'nombre':nombre,'ventas1':ventas1,'ventas':ventas}, ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1923074529.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'nombre':nombre,'ventas1':ventas1,'ventas':ventas}, ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1923074529.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'nombre':nombre,'ventas1':ventas1,'ventas':ventas}, ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_6000\1764769722.py:14: FutureWarning: The frame.append method is deprecated and will b

Proceso Finaliza 2022-11-08 19:11:04.609173
